In [31]:
import cv2
import mediapipe as mp
import numpy as np
import math
import time
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.layers import Conv2D,MaxPooling2D,BatchNormalization
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping,ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.models import model_from_json
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

In [23]:
num_features = 64
num_labels = 7
epochs = 100
width, height = 48, 48

model = Sequential()

model.add(Conv2D(filters = num_features, kernel_size = (3,3), activation = "relu",
                 input_shape = (width,height,1), data_format = "channels_last", kernel_regularizer = l2(0.01)))
model.add(Conv2D(filters = num_features, kernel_size = (3,3), activation = "relu", padding = "same"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))
model.add(Dropout(rate = 0.5))

model.add(Conv2D(filters = 2*num_features, kernel_size = (3,3), activation = "relu",padding = "same"))
model.add(BatchNormalization())
model.add(Conv2D(filters = 2*num_features, kernel_size = (3,3), activation = "relu",padding = "same"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))
model.add(Dropout(rate = 0.5))

model.add(Conv2D(filters = 2*2*num_features, kernel_size = (3,3), activation = "relu",padding = "same"))
model.add(BatchNormalization())
model.add(Conv2D(filters = 2*2*num_features, kernel_size = (3,3), activation = "relu",padding = "same"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))
model.add(Dropout(rate = 0.5))

model.add(Conv2D(filters = 2*2*2*num_features, kernel_size = (3,3), activation = "relu",padding = "same"))
model.add(BatchNormalization())
model.add(Conv2D(filters = 2*2*2*num_features, kernel_size = (3,3), activation = "relu",padding = "same"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))
model.add(Dropout(rate = 0.5))

model.add(Flatten())

model.add(Dense(2*2*2*num_features,activation = "relu"))
model.add(Dropout(rate = 0.4))
model.add(Dense(2*2*num_features,activation = "relu"))
model.add(Dropout(rate = 0.4))
model.add(Dense(2*num_features,activation = "relu"))
model.add(Dropout(rate = 0.5))

model.add(Dense(num_labels,activation = "softmax"))

model.load_weights("Data/model_01_expressions.h5")

In [150]:
class FaceDetector:
    
    def __init__(self,minDetection = 0.5,modelSelection = 0):
        
        self.minDetection = minDetection
        self.modelSelection = modelSelection
        
        self.mpDraw = mp.solutions.drawing_utils
        self.mpFace = mp.solutions.face_detection
        
        self.Face = self.mpFace.FaceDetection(self.minDetection,self.modelSelection)
        
        
    def findFaces(self,image,draw = True, color = (0,255,0)):
        
        imageRGB = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        self.results = self.Face.process(imageRGB)
        h,w,c = image.shape
        bboxs = []
        
        if self.results.detections:
            #print(self.results.detections)
            for _id,detection in enumerate(self.results.detections):
                
                bboxC = detection.location_data.relative_bounding_box
                bbox = int(bboxC.xmin*w),int(bboxC.ymin*h),int(bboxC.width*w),int(bboxC.height*h)
                bboxs.append([_id,bbox,detection.score])
                
                if draw:
                    
                    cv2.rectangle(image,(bbox[0],bbox[1]),(bbox[0]+bbox[2],bbox[1] + bbox[3]),color,2)
                    
                    cv2.putText(image,f"Face Score : {int(detection.score[0]*100)}%",(bbox[0]-10,bbox[1]-20),
                               cv2.FONT_HERSHEY_PLAIN,2,color,2)
                    
        return image,bboxs
    
    def fancyDraw(self,image,bbox,lenght = 50,thickness = 10, color = (0,255,0)):
        
        x,y,w,h = bbox
        x1,y1 = x+w,y+h
        
        cv2.line(image,(x,y),(x+lenght,y),color,thickness)
        cv2.line(image,(x,y),(x,y+lenght),color,thickness)
        
        cv2.line(image,(x,y1),(x+lenght,y1),color,thickness)
        cv2.line(image,(x,y1),(x,y1-lenght),color,thickness)
        
        cv2.line(image,(x1,y),(x1-lenght,y),color,thickness)
        cv2.line(image,(x1,y),(x1,y+lenght),color,thickness)
        
        cv2.line(image,(x1,y1),(x1-lenght,y1),color,thickness)
        cv2.line(image,(x1,y1),(x1,y1-lenght),color,thickness)
        
        return image
       
        
class Bar:
    def __init__(self,pos,width,height,text):
        self.width = width
        self.height = height
        self.pos = pos
        
    def draw(self,image):
        
        cv2.rectangle(image,self.pos,(self.pos[0] + self.width, self.pos[1] + self.height),
                     (0,0,0),3) 
        return image
        
    def inside(self,image,rate,text):
        
        cv2.rectangle(image,self.pos,(self.pos[0] + rate, self.pos[1] + self.height),(200,250,20),-1) 
        
        cv2.putText(image,text,(self.pos[0] + 70, self.pos[1] + 35),cv2.FONT_HERSHEY_PLAIN,1.3,(0,0,0),2)
        
        return image

        
pTime = 0
def fps():
    global pTime
    cTime = time.time()
    fps = 1 / (cTime-pTime)
    pTime = cTime
    return fps
       
cap = cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,720)

face_detector =FaceDetector()

BarList = []
for i in range(7):
    xpos = 10
    ypos = 60*i + 10
    BarList.append(Bar((xpos,ypos),300,50,"{}%"))
    

expressions = ['Anger', 'Disgusted', 'Fear', 'Happy', 'Sad', 'Surprised', 'Neutral']

while True:

    conn,frame = cap.read()
    frame = cv2.flip(frame,1)
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

    for Bar in BarList:
        frame = Bar.draw(frame)
    

    frame,bboxs = face_detector.findFaces(frame,draw=True,color=(0,0,255))

    if bboxs:
        info = bboxs[0]
        _id = info[0]
        bbox = info[1]
        score = info[2]
        frame = face_detector.fancyDraw(frame,bbox)
        x,y,w,h = bbox
        try:
            roi = gray[y:y+h,x:x+w]
            roi = roi.astype("float") / 255.0
            resize = cv2.resize(roi,(48,48))
            cropped_img = np.expand_dims(np.expand_dims(resize,-1), 0)
            prediction = model.predict(cropped_img)[0]

            label = expressions[np.argmax(prediction)]
            cv2.putText(frame,"Emotion : {}".format(label),(bbox[0]-10,bbox[1]-50),
                        cv2.FONT_HERSHEY_PLAIN,2,(0,0,255),2)

            if prediction is not None:
                for emotion,prob,Bar in zip(expressions,prediction,BarList):
                    text = "{}: {:.2f}%".format(emotion,prob*100)

                    barPer = np.interp(prob*100,(0,100),(10,300))
                    frame = Bar.inside(frame,int(barPer),text)
        except:
            pass
    else:
        
        for emotion,Bar in zip(expressions,BarList):
            text = "{}: {:.2f}%".format(emotion,0)
            frame = Bar.inside(frame,1,text)

    fps_=fps()
    cv2.putText(frame,f"FPS : {int(fps_)}",(frame.shape[1]-250,70),cv2.FONT_HERSHEY_PLAIN,3,(255,0,0),3)
    cv2.imshow("Face",frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()